<a href="https://colab.research.google.com/github/archiebenn/BIOLM0050_kaggle/blob/master/protein_embedding_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q torch transformers sentencepiece h5py

In [21]:
import h5py
import numpy as np
import pandas as pd
import os
import torch

from transformers import T5EncoderModel, T5Tokenizer

## Load ProtT5

In [30]:
# set tokeniser
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokeniser = T5Tokenizer.from_pretrained(
    "Rostlab/prot_t5_xl_uniref50",
    do_lower_case=False
)

model = T5EncoderModel.from_pretrained(
    "Rostlab/prot_t5_xl_uniref50"
).to(device)

model.eval()

RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-6989fafe-0689fac933d9078b30c57dfd;1735d15c-d232-4c97-99a5-917c0464d979)

Repository Not Found for url: https://huggingface.co/api/models/Rostlab/prot_t5_large_uniref50/tree/main/additional_chat_templates?recursive=false&expand=false.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated. For more details, see https://huggingface.co/docs/huggingface_hub/authentication
Invalid username or password.

In [18]:
# import training csv
from google.colab import files
uploaded = files.upload()


Saving train.csv to train (1).csv


## Embed the protein sequences in train.csv

In [15]:
def embed_protein(seq):

  # reformat sequences to be space separated AAs:
  seq = ' '.join(list(seq))

  inputs = tokenizer(seq, return_tensors="pt")
  inputs = {k: v.to(device) for k, v in inputs.items()}

  with torch.no_grad():
    outputs = model(**inputs)

  emb = outputs.last_hidden_state.squeeze(0).cpu().numpy()

  return emb.mean(axis=0)

In [24]:
# data setup
df_train = pd.read_csv("train.csv")

print(f'Train DF length: {len(df_train)}')
df_train.head()


Train DF length: 16077


,Id,acc,partition,cytoplasm,nucleus,extracellular,cell_surface,mitochondrion,endom,sequence,...,aa_frac_M,aa_frac_N,aa_frac_P,aa_frac_Q,aa_frac_R,aa_frac_S,aa_frac_T,aa_frac_V,aa_frac_W,aa_frac_Y
0,0,P61966,0,0,0,0,0,0,1,MMRFMLLFSRQGKLRLQKWYLATSDKERKKMVRELMQVVLARKPKM...,...,0.051,0.013,0.013,0.044,0.063,0.057,0.019,0.063,0.013,0.044
1,1,Q9VTK2,0,0,0,0,0,0,1,MSATYTNTITQRRKTAKVRQQQQHQWTGSDLSGESNERLHFRSRST...,...,0.028,0.032,0.044,0.043,0.068,0.080,0.063,0.059,0.025,0.038
2,2,O95858,3,0,0,0,1,0,1,MPRGDSEQVRYCARFSYLWLKFSLIIYSTVFWLIGALVLSVGIYAE...,...,0.034,0.041,0.031,0.027,0.044,0.044,0.051,0.078,0.014,0.058
3,3,Q9WUX5,0,1,0,0,0,0,1,MGRSLTCPFGISPACGAQASWSIFGVGTAEVPGTHSHSNQAAAMPH...,...,0.023,0.036,0.089,0.051,0.050,0.117,0.044,0.058,0.008,0.011
4,4,Q9NQC3-3,1,0,0,0,0,0,1,MDGQKKNWKDKVVDLLYWRDIKKTGVVFGASLFLLLSLTVFSIVSV...,...,0.015,0.030,0.015,0.035,0.035,0.070,0.035,0.101,0.015,0.040


In [27]:
# keep just the Id and sequence in the data frame
df_seq_id = df_train[["Id", "sequence"]]
df_seq_id.head()

,Id,sequence
0,0,MMRFMLLFSRQGKLRLQKWYLATSDKERKKMVRELMQVVLARKPKM...
1,1,MSATYTNTITQRRKTAKVRQQQQHQWTGSDLSGESNERLHFRSRST...
2,2,MPRGDSEQVRYCARFSYLWLKFSLIIYSTVFWLIGALVLSVGIYAE...
3,3,MGRSLTCPFGISPACGAQASWSIFGVGTAEVPGTHSHSNQAAAMPH...
4,4,MDGQKKNWKDKVVDLLYWRDIKKTGVVFGASLFLLLSLTVFSIVSV...


In [55]:
# testing for longest sequence in train.csv:
# Compute string lengths
df_train["sequence"] = df_train["sequence"].astype(str)


In [31]:
# write out the .h5 file:

with h5py.File("train_embeddings.h5", "w") as f:
  for _, row in df_seq_id.iterrows():

    # 1. run embed protein function on each sequence:
    seq = row["sequence"]
    emb = embed_protein(seq)

    # 2. set prot_id as a string for h5py to work (for h5 file column)
    prot_id = str(row["Id"])

    # 3. add embedded protein to h5 file
    f.create_dataset(prot_id, data=emb)

    # 4. free up GPU memory from this iteration:
    del emb
    torch.cuda.empty_cache()

OutOfMemoryError: CUDA out of memory. Tried to allocate 570.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 264.12 MiB is free. Process 2713 has 14.48 GiB memory in use. Of the allocated memory 13.90 GiB is allocated by PyTorch, and 468.85 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)